In [35]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

features_names = ['keyword', 'location', 'cant_palabras', 'palabras_unicas', 'cant_stop_words', 'cant_url', 'prom_long_palabra', 'cant_puntuacion', 'cant_hashtag', 'cant_mencion',
'cant_signos_pregunta', 'cant_signos_exclamacion', 'subjetividad', 'cant_oraciones', 'cant_apariciones_keyword', 'cant_numeros', 'cant_minusculas', 'cant_mayusculas', 'cant_vocales', 'cant_consonant']

train = pd.read_csv('train_features.csv')
test = pd.read_csv('test_features.csv')

X_train = train.copy()

keyword = list(train['keyword'])
le = preprocessing.LabelEncoder()
keyword_codificada = le.fit_transform(keyword)
X_train['keyword'] = keyword_codificada

location = list(train['location'])
le = preprocessing.LabelEncoder()
location_codificada = le.fit_transform(location)
X_train['location'] = location_codificada

X_train = X_train.drop(['target', 'id', 'text'], axis=1)

In [36]:
model = XGBClassifier()
model.fit(X_train, train.target)

thresholds = model.feature_importances_
for thresh in thresholds:
	selection = SelectFromModel(model, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)

In [37]:
for features in zip(features_names, thresholds):
  print(features)

('keyword', 0.04617176)
('location', 0.02940135)
('cant_palabras', 0.031083787)
('palabras_unicas', 0.016288623)
('cant_stop_words', 0.0017762581)
('cant_url', 0.019211877)
('prom_long_palabra', 0.23900564)
('cant_puntuacion', 0.06783052)
('cant_hashtag', 0.023148706)
('cant_mencion', 0.015206396)
('cant_signos_pregunta', 0.05404374)
('cant_signos_exclamacion', 0.02217784)
('subjetividad', 0.03980465)
('cant_oraciones', 0.035365842)
('cant_apariciones_keyword', 0.098420925)
('cant_numeros', 0.033310015)
('cant_minusculas', 0.057847843)
('cant_mayusculas', 0.03490728)
('cant_vocales', 0.048646104)
('cant_consonant', 0.046943203)


Elegimos: 'keyword', 'prom_long_palabra', 'cant_puntuacion', 'cant_signos_pregunta', 'cant_apariciones_keyword', 'cant_minusculas', 'cant_vocales'

